# Use inceptionv4 and multiple input

# 1. Data Clean

## Define data clean function
if you need to do zero center, you should use data_normal rather than data_clean.

In [ ]:
def data_normal(X_,y_,is_training):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images_1 = []
    images_2 = []
    avergae_bright=[]
    for i, row in X_.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        # Rescale
        hh = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        hv = (band_2 - band_2.min()) / (band_2.max() - band_2.min())

        images_1.append(hh)
        images_2.append(hv)
        
    images_1=np.array([band[:,:,np.newaxis] for band in images_1])
    images_2=np.array([band[:,:,np.newaxis] for band in images_2])
    X_data_angle=np.reshape(np.sin(X_["inc_angle"]),(X_["inc_angle"].shape[0],1))
#     X_data_ave=np.reshape(avergae_bright,(len(avergae_bright),1))
#     X_data_plus=np.concatenate([X_data_angle[:, np.newaxis], 
#                               X_data_ave[:, np.newaxis]], 
#                               axis=2)
    X_data_plus=np.concatenate([X_data_angle[:, np.newaxis]], 
                              axis=2)
    X_data_plus=X_data_plus.reshape(X_data_plus.shape[0],X_data_plus.shape[2])
    if is_training:
        y_data = keras.utils.to_categorical(y_, num_classes)
    else:
        y_data = None
    return (images_1,images_2,X_data_plus,y_data)

# Define threadsafe_generator
# If we want to use data augmentation in training process,we have to use @threadsafe_generator to make any generator thread safe.

# Note: A non thread safe generator in a multithreaded envrionment just crashes, yieldsing a 'ValueError: generator already executing' error.You can try it yourself by removing the @threadsafe_generator decorator from count().

# Also, since we have to use multiple generators to feed both the image data and the angle data, we have to define our own generator: generator_img_angle()

import threading
from keras.preprocessing.image import ImageDataGenerator

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()
 
    def __iter__(self):
        return self
 
    def __next__(self):
        with self.lock:
            return self.it.__next__()
 
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g
 
#Define data clean function
# if you need to do zero center, you should use data_normal rather than data_clean.

import matplotlib.pyplot as plt
def plt_images(X_images):
    plt.imshow(X_images,cmap = plt.cm.gray)

def data_normal(X_,y_,is_training):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images_1 = []
    images_2 = []
    avergae_bright=[]
    index=1
    for i, row in X_.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
#         avei=(np.sum(band_1)+np.sum(band_2))/(band_1.shape[0]+band_2.shape[0])
        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        
        images_1.append(r[:,:,np.newaxis])
        images_2.append(g[:,:,np.newaxis])
#         avergae_bright.append(avei)
        
    X_data_angle=np.reshape(np.sin(X_["inc_angle"]),(X_["inc_angle"].shape[0],1))
    X_data_plus=np.concatenate([X_data_angle[:, np.newaxis]], 
                              axis=2)
    X_data_plus=X_data_plus.reshape(X_data_plus.shape[0],X_data_plus.shape[2])
    if is_training:
        y_data = keras.utils.to_categorical(y_, num_classes)
    else:
        y_data = None

    return (np.array(images_1),np.array(images_2),X_data_plus,y_data)


# Define threadsafe_generator
# If we want to use data augmentation in training process,we have to use @threadsafe_generator to make any generator thread safe.

# Note: A non thread safe generator in a multithreaded envrionment just crashes, yieldsing a 'ValueError: generator already executing' error.You can try it yourself by removing the @threadsafe_generator decorator from count().

# Also, since we have to use multiple generators to feed both the image data and the angle data, we have to define our own generator: generator_img_angle()
import threading
from keras.preprocessing.image import ImageDataGenerator

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()
 
    def __iter__(self):
        return self
 
    def __next__(self):
        with self.lock:
            return self.it.__next__()
 
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g
 
    
@threadsafe_generator
def generator_img_plus_new( generator, X1,X2,X_plus, y, batch_size = 32 ):
    SEED=816
    generator_seed = np.random.randint( SEED )
    gen_X1 = generator.flow( X1, y, 
                             batch_size = batch_size, seed = generator_seed )
    gen_X2 = generator.flow( X1, X2, 
                             batch_size = batch_size, seed = generator_seed )
    gen_X3 = generator.flow( X1, X_plus, 
                             batch_size = batch_size, seed = generator_seed )

    while True:
        X1i = gen_X1.next()
        X2i = gen_X2.next()
        X3i = gen_X3.next()
        yield [ X1i[0], X2i[1],X3i[1] ], X1i[1]
    
#load data
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import keras
import math
#Load the data.
train = pd.read_json("../input/train.json")

#use mean of angle to replace "na"
inc_angle = train.inc_angle.replace('na',0)
idx=np.where(inc_angle==0)
inc_angle = inc_angle.drop(idx[0])
inc_mean=np.mean(inc_angle)
train.inc_angle = train.inc_angle.replace('na',inc_mean)


#Do Data Cleaning on training data and validation data
num_classes=2


y_train=train['is_iceberg']

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(train,y_train, random_state=1, train_size=0.75)
# X_train_cv,X_train_angle,y_train_cv=data_clean(X_train_cv,y_train_cv,is_training=True)
# X_valid,X_valid_angle,y_valid=data_clean(X_valid,y_valid,is_training=True)

X_train_1,X_train_2,X_train_plus,y_train_cv=data_normal(X_train_cv,y_train_cv,is_training=True)
X_valid_1,X_valid_2,X_valid_plus,y_valid=data_normal(X_valid,y_valid,is_training=True)

print(X_train_1.shape,X_train_2.shape,X_train_plus.shape,y_train_cv.shape)
print(X_valid_1.shape,X_valid_2.shape,X_valid_plus.shape,y_valid.shape)
        
        
#load data
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split
import pandas as pd
import json
import keras
import math
#Load the data.
train = pd.read_json("../input/train.json")

#use mean of angle to replace "na"
inc_angle = train.inc_angle.replace('na',0)
idx=np.where(inc_angle==0)
inc_angle = inc_angle.drop(idx[0])
inc_mean=np.mean(inc_angle)
train.inc_angle = train.inc_angle.replace('na',inc_mean)
# train=train.drop(idx[0])
# train.inc_angle = train.inc_angle.replace('na',0)
    
                      
#Do Data Cleaning on training data and validation data                   
num_classes=2


y_train=train['is_iceberg']

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(train,y_train, random_state=1, train_size=0.75)
# X_train_cv,X_train_angle,y_train_cv=data_clean(X_train_cv,y_train_cv,is_training=True)
# X_valid,X_valid_angle,y_valid=data_clean(X_valid,y_valid,is_training=True)

X_train_1,X_train_2,X_train_plus,y_train_cv=data_normal(X_train_cv,y_train_cv,is_training=True)
X_valid_1,X_valid_2,X_valid_plus,y_valid=data_normal(X_valid,y_valid,is_training=True)

print(X_train_1.shape,X_train_2.shape,X_train_plus.shape,y_train_cv.shape)
print(X_valid_1.shape,X_valid_2.shape,X_valid_plus.shape,y_valid.shape)


/home/cv/miniconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/home/cv/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


(1203, 75, 75, 1) (1203, 75, 75, 1) (1203, 1) (1203, 2)
(401, 75, 75, 1) (401, 75, 75, 1) (401, 1) (401, 2)
(1203, 75, 75, 1) (1203, 75, 75, 1) (1203, 1) (1203, 2)
(401, 75, 75, 1) (401, 75, 75, 1) (401, 1) (401, 2)


# 2. Train model

If we use data augmentaion, use the model.fit_generator version, otherwise use model.fit

In [ ]:
from keras.applications.vgg16 import VGG16

from keras.applications.densenet import DenseNet
from keras.preprocessing import image
from keras.models import Model
from keras.models import load_model 
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D,Dense, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras import regularizers
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
import tensorflow as tf

import math
import os
import keras

#Macro defined
batch_size =32
num_classes =2
epochs =300
batch_num_per_epoch=math.ceil(X_train_cv.shape[0]/batch_size)
batch_num_val=math.ceil(X_valid.shape[0]/batch_size)
data_augmentation = True
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_iceberg_transe.h5'
filepath="keras_iceberg_transe_epoch_mymodel.h5"


def dense_block(inputs,filter_num):
    x=Dense(filter_num,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.1))(inputs)
    x=keras.layers.advanced_activations.LeakyReLU(alpha=0.01)(x)
    x=Dropout(0.4)(x)
    return x

def model_i(input_x,angle_input):
    image_out= GlobalAveragePooling2D()(input_x)
    x = keras.layers.concatenate([image_out, angle_input],axis=-1)
    x=dense_block(x,256)
    x=dense_block(x,16)
    prediction = Dense(num_classes, activation='selu',kernel_initializer='glorot_normal',
                   kernel_regularizer=regularizers.l2(0.001))(x)
    return prediction
# create the base pre-trained model

# #dense_net
# base_model=DenseNet(blocks=[6, 12, 48, 32],
#              include_top=False,
#              weights='imagenet',
#              input_tensor=None,
#              input_shape=None,
#              pooling=None,
#              classes=1000)
# x= base_model.output

#inception_v4
# base_model_1=keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=False,weights=None, 
#                                                                       input_tensor=None, 
#                              input_shape=(75,75,1), pooling=None, classes=1000)
# x1= base_model_1.get_layer('mixed_7a').output

# base_model_2=keras.applications.inception_resnet_v2_change.InceptionResNetV2(include_top=False,weights=None, input_tensor=None, 
#                              input_shape=None, pooling=None, classes=1000)
# x2= base_model_2.get_layer('mixed_7a_2').output

#inception_v3
base_model_1=keras.applications.inception_v3.InceptionV3(include_top=False,
                                            weights=None,
                                            input_tensor=None,
                                            input_shape=(75,75,1),
                                            pooling=None,
                                            classes=1000)
main_input1= base_model_1.get_layer('mixed3').output
base_model_2=keras.applications.inception_v3_change.InceptionV3(include_top=False,
                                            weights=None,
                                            input_tensor=None,
                                            input_shape=(75,75,1),
                                            pooling=None,
                                            classes=1000)
main_input2= base_model_2.get_layer('mixed2_2').output

#vgg16
# base_model=keras.applications.vgg16.VGG16(include_top=False,weights='imagenet',
#                                 input_tensor=None, input_shape=None,
#                                 pooling=None,
#                                 classes=1)
# x= base_model.get_layer('block4_pool').output

#vgg-19
# base_model=keras.applications.vgg19.VGG19(include_top=False, weights='imagenet',
#                                 input_tensor=None, input_shape=None,
#                                 pooling=None,
#                                 classes=1000)
# x= base_model.get_layer('block5_pool').output

#xception
# base_model=keras.applications.xception.Xception(include_top=False, weights='imagenet',
#                                     input_tensor=None, input_shape=None,
#                                     pooling=None, classes=1000)
# x= base_model.get_layer('block14_sepconv1_act').output

#resnet-50
# base_model=keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet',
#                                 input_tensor=None, input_shape=None,
#                                 pooling=None,
#                                 classes=1000)
# x= base_model.get_layer('avg_pool').output

plus_input = Input(shape=(1,), name='plus_input')
x1_pre=model_i(main_input1,plus_input)
x2_pre=model_i(main_input2,plus_input)

x_pre=keras.layers.concatenate([x1_pre,x2_pre])
x_pre=dense_block(x_pre,16)
predictions = Dense(num_classes, activation='softmax',name='main_output',
                    kernel_regularizer=regularizers.l2(0.001),
                    kernel_initializer='glorot_normal')(x1_pre)


# this is the model we will train
model = Model(inputs=[base_model_1.input,base_model_2.input, plus_input], outputs=predictions)

# compile the model (should be done *after* setting layers to non-trainable)
opt=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

#input data

# Fit the model on the batches generated by datagen.flow().
# model.fit([X_train_1,X_train_2, X_train_plus],y_train_cv,
#           validation_data=([X_valid_1,X_valid_2,X_valid_plus],y_valid),
#           epochs=epochs, batch_size=batch_size,
#           callbacks=[keras.callbacks.ModelCheckpoint(filepath,
#                                                         monitor='val_acc',
#                                                         verbose=0,
#                                                         save_best_only=True, 
#                                                         mode='auto')])

image_augmentation = ImageDataGenerator( rotation_range = 20,
                                         horizontal_flip = True,
                                         vertical_flip = True,
                                         width_shift_range = .3,
                                         height_shift_range =.3,
                                         zoom_range = .1 )

train_generator = generator_img_plus_new( image_augmentation, X_train_1,X_train_2,
                                X_train_plus, y_train_cv, 
                                batch_size = batch_size)
model.fit_generator(train_generator,epochs=epochs,
                    validation_data=([X_valid_1,X_valid_2,X_valid_plus],y_valid),
#                   validation_data=generator_img_angle(X_valid,X_valid_angle,y_valid,
#                                                       batch_size,is_training=True),
                    validation_steps=batch_num_val,
                    workers=4,
                    steps_per_epoch=batch_num_per_epoch,
                    # 该回调函数将在每个epoch后保存模型到filepath
                    callbacks=[keras.callbacks.ModelCheckpoint(filepath,
                                                               monitor='val_acc',
                                                               verbose=0,
                                                               save_best_only=True, 
                                                               mode='auto')])

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate([X_valid_1,X_valid_2, X_valid_plus],y_valid, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/300


## validation check

In [3]:
import keras
# Score trained model.
scores = model.evaluate([X_valid_1,X_valid_2, X_valid_plus],y_valid, verbose=1)
# batch_size=32
# batch_num_val=math.ceil(X_valid.shape[0]/batch_size)
# scores = model.evaluate_generator(generator_img_plus(X_valid, X_valid_plus, y_valid,batch_size=batch_size),
#                                   steps=batch_num_val)

print("validation accuracy:",scores)

401/401 [==============================] - 1s 3ms/step
validation accuracy: [0.71279475397599901, 0.85536159660453515]


# 3.Predict

## Load a pretrained model

In [1]:
from keras.models import load_model
model_path="../iceberg_code/keras_iceberg_transe_epoch_mymodel.h5"
model = load_model(model_path)  

Using TensorFlow backend.


## predict

In [23]:
import json
import numpy as np
import pandas as pd
import math

cut_set_num=10
lenth_test=8424
num_classes=2
is_training=False
test_path="../input/test_cut/"
batch_size=32


predicted_test=[]
result_max=[]
test_id=[]


for ci in range(cut_set_num):
    test_pathi=test_path+"test%d.json"%ci
    test_json_str = open(test_pathi, 'r').readlines()[0]
    test=pd.read_json(test_json_str)
    
    #use mean of angle to replace "na"
    inc_angle = test.inc_angle.replace('na',0)
    idx=np.where(inc_angle==0)
    inc_angle = inc_angle.drop(idx[0])
    inc_mean=np.mean(inc_angle)
    test.inc_angle = test.inc_angle.replace('na',inc_mean)
#     test= test.drop(idx[0])
#     test.inc_angle = test.inc_angle.replace('na',0)
   
    X_test,X_test_plus,y_test=data_normal(X_=test,y_=None,is_training=False)
    batch_num_predic=math.ceil(X_test.shape[0]/batch_size)
#     print(X_test.shape[0],batch_num_predic)
    #发现一个问题,steps总是比我设置的多出10
    predicted_testi=model.predict([X_test,X_test_plus])
    
#     predicted_testi=model.predict_generator(generator_img_angle(X11=X_test,
#                                                                 X2=X_test_angle, 
#                                                                 Y=None,
#                                                                 batch_size=batch_size,
#                                                                 is_training=False),
#                                             steps=batch_num_predic)
    
    test_id.extend(test["id"])
    predicted_testi=predicted_testi.reshape(predicted_testi.shape[0],2)
    for index in range(len(predicted_testi)):
        predicted_test.append(predicted_testi[index][1])
    result_max.extend( np.argmax(predicted_testi, axis = 1) )
    
    print("batch%d has finished!"%ci)
    print()

/home/cv/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


batch0 has finished!

batch1 has finished!

batch2 has finished!

batch3 has finished!

batch4 has finished!

batch5 has finished!

batch6 has finished!

batch7 has finished!

batch8 has finished!

batch9 has finished!



In [24]:
# save result
submission = pd.DataFrame()
print(len(test_id),len(result_max),len(predicted_test))
submission['id']=test_id
submission['is_iceberg']=result_max
submission.to_csv('sub.csv', index=False)
# save probablity

submission['is_iceberg']=predicted_test
submission.to_csv('plus_inceptionv3.csv', index=False,float_format="%.6lf")


8424 8424 8424


In [67]:
print(base_model_1.summary())
from keras.utils import plot_model
plot_model(base_model_1, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_895 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_815 (BatchN (None, None, None, 3 96          conv2d_895[0][0]                 
__________________________________________________________________________________________________
activation_893 (Activation)     (None, None, None, 3 0           batch_normalization_815[0][0]    
__________________________________________________________________________________________________
conv2d_896

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.